In [ ]:
!pip install transformers
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import LongformerTokenizer, LongformerForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from transformers import LongformerTokenizer, LongformerModel, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

In [ ]:
def load_training_data(data_file):
    df = pd.read_excel(data_file)
    df = df.dropna()
    texts = df['text'].tolist()
    content_mapping = {'Holding Response': 0, 'Closing Response': 1, 'Asking for users details': 2, 'Response to details shared': 3, 'Responded to previous post': 4, 'Unclear concern': 5, 'Helpline number/Mail': 6, 'Asking again the user details': 7, 'Showing interest in service': 8}
    labels = [content_mapping.get(content, 1) for content in df['class'].tolist()]
    return texts, labels

In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')
data_file = "/content/drive/MyDrive/Colab/output_amend_tata.xlsx"
texts, labels = load_training_data(data_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def print_label_counts(labels):
    label_counts = {}
    for label in labels:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1
    sorted_labels = sorted(label_counts.keys())
    for label in sorted_labels:
        print(f"{label}: {label_counts[label]}")

print_label_counts(labels)

0: 6104
1: 6445
2: 4180
3: 3333
4: 672
5: 3853
6: 1703
7: 869
8: 4791


In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        try:
            encoding = self.tokenizer(
                text,
                return_tensors='pt',
                max_length=self.max_length,
                padding='max_length',
                truncation=True
            )
            return {
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'label': torch.tensor(label)
            }
        except Exception as e:
            print(f"Error processing example at index {idx}.")
            print(f"Text: {text}")
            print(f"Label: {label}")
            raise e

In [ ]:
class LongformerClassifier(nn.Module):
    def __init__(self, longformer_model_name, num_classes):
        super(LongformerClassifier, self).__init__()
        self.longformer = LongformerModel.from_pretrained(longformer_model_name, from_tf=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(self.longformer.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.longformer(input_ids=input_ids, attention_mask=attention_mask).last_hidden_state
        pooled_output = outputs.mean(dim=1)  # Adjust pooling strategy as needed
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [ ]:
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # if text is None or label is None:
        #   raise ValueError(f"Found a sample with None value: text={text}, label={label}")
        if text is None or label is None:
          raise ValueError(f"Found a sample with None value at index {idx}: text={text}, label={label}")
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [ ]:
# !pip install logging
# from tqdm import tqdm
# def train_longformer(model, data_loader, optimizer, scheduler, device):
#     model.train()
#     total_batches = len(data_loader)
#     print(total_batches)
#     # for batch in data_loader:
#     #     optimizer.zero_grad()
#     #     input_ids = batch['input_ids'].to(device)
#     #     attention_mask = batch['attention_mask'].to(device)
#     #     labels = batch['label'].to(device)
#     #     outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#     #     loss = nn.CrossEntropyLoss()(outputs, labels)
#     #     loss.backward()
#     #     optimizer.step()
#     #     scheduler.step()
#     for batch_num, batch in enumerate(data_loader, 1):  # Start from 1 for better printing
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         loss = nn.CrossEntropyLoss()(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         scheduler.step()

#         # Print training progress
#         if batch_num % (total_batches // 10) == 0:  # Print every 10%
#             progress_percent = (batch_num / total_batches) * 100
#             print(f"Batch [{batch_num}/{total_batches}] - Loss: {loss.item():.4f} - Progress: {progress_percent:.2f}%")

#     print("Training completed.")

In [ ]:
!pip install tqdm
from tqdm import tqdm

def train_longformer(model, data_loader, optimizer, scheduler, device):
    model.train()
    total_batches = len(data_loader)
    print(f"Total batches: {total_batches}")
    total_loss = 0.0

    with tqdm(total=total_batches, desc="Training Progress", unit="batch") as pbar:
        for batch_num, batch in enumerate(data_loader, 1):
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            loss.backward()
            optimizer.step()
            scheduler.step()

            total_loss += loss.item()

            # Update progress bar
            pbar.update(1)
            pbar.set_postfix({"Loss": loss.item()})

            # Print training progress
            if batch_num % (total_batches // 10) == 0:  # Print every 10%
                progress_percent = (batch_num / total_batches) * 100
                print(f"Batch [{batch_num}/{total_batches}] - Loss: {loss.item():.4f} - Progress: {progress_percent:.2f}%")

    # Calculate and print average loss
    average_loss = total_loss / total_batches
    print(f"Training completed. Average Loss: {average_loss:.4f}")

In [ ]:
def evaluate_longformer(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions)

In [ ]:
accumulation_steps = 8  # Adjust as needed
num_classes = 9
max_length = 4096
batch_size = 4
num_epochs = 10
learning_rate = 2e-5

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2, random_state=42)

In [ ]:
from transformers import LongformerModel, LongformerTokenizer
from google.colab import userdata
longformer_model_name = "allenai/longformer-base-4096"
tokenizer = LongformerTokenizer.from_pretrained(longformer_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LongformerClassifier(longformer_model_name, num_classes).to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate)
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer, max_length)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/765M [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing LongformerModel.



In [ ]:
import time
import pytz
from datetime import datetime
torch. cuda. empty_cache()
ist_timezone = pytz.timezone('Asia/Kolkata')
save_path = "/content/drive/MyDrive/Colab/models/"
for epoch in range(num_epochs):
  print(f"Epoch {epoch + 1}/{num_epochs}")
  start_time = time.time()
  start_time_ist = datetime.fromtimestamp(start_time, ist_timezone)
  print(f"Epoch {epoch + 1} training started at: {start_time_ist.strftime('%a %b %d %I:%M %p %Y')}")
  train_longformer(model, train_dataloader, optimizer, scheduler, device)
  #torch.save(model.state_dict(), f"model_longformer_training_data_epoch{epoch+1}.pt")  # Save the model
  torch.save(model.state_dict(), f"{save_path}model_longformer_training_data_epoch{epoch+1}.pt")  # Save the model
  print(f"Epoch [{epoch+1}/{num_epochs}] - Model saved to {save_path}model_longformer_training_data_epoch{epoch+1}.pt")
  end_time = time.time()
  end_time_ist = datetime.fromtimestamp(end_time, ist_timezone)
  epoch_time = end_time - start_time
  minutes, seconds = divmod(int(epoch_time), 60)
  print(f"Epoch {epoch + 1} training ended at: {end_time_ist.strftime('%a %b %d %I:%M %p %Y')}")
  print(f"Time taken for epoch {epoch+1}: {minutes} minutes and {seconds} seconds")
  accuracy, report = evaluate_longformer(model, val_dataloader, device)
  print(f"Validation Accuracy: {accuracy:.4f}")
  print(report)
# Add gradient accumulation steps
# accumulation_steps = 8  # Adjust as needed

# # Training loop
# for epoch in range(num_epochs):
#     print(f"Epoch {epoch + 1}/{num_epochs}")
#     start_time = time.time()
#     start_time_ist = datetime.fromtimestamp(start_time, ist_timezone)
#     print(f"Epoch {epoch + 1} training started at: {start_time_ist.strftime('%a %b %d %I:%M %p %Y')}")
#     model = model.half()
#     model.train()
#     optimizer.zero_grad()

#     for batch_num, batch in enumerate(train_dataloader):
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)
#         outputs = model(input_ids=input_ids, attention_mask=attention_mask)
#         loss = nn.CrossEntropyLoss()(outputs, labels)
#         loss = loss / accumulation_steps  # Divide the loss by accumulation steps
#         loss.backward()

#         if (batch_num + 1) % accumulation_steps == 0:
#             optimizer.step()
#             scheduler.step()
#             optimizer.zero_grad()

#     end_time = time.time()
#     end_time_ist = datetime.fromtimestamp(end_time, ist_timezone)
#     epoch_time = end_time - start_time
#     minutes, seconds = divmod(int(epoch_time), 60)
#     print(f"Epoch {epoch + 1} training ended at: {end_time_ist.strftime('%a %b %d %I:%M %p %Y')}")
#     print(f"Time taken for epoch {epoch + 1}: {minutes} minutes and {seconds} seconds")

#     # Validation
#     model.eval()
#     accuracy, report = evaluate_longformer(model, val_dataloader, device)
#     print(f"Validation Accuracy: {accuracy:.4f}")
#     print(report)

#     # Save the model
#     torch.save(model.state_dict(), f"model_longformer_training_data_epoch{epoch+1}.pt")
#     print(f"Epoch [{epoch+1}/{num_epochs}] - Model saved.")

# #End of training loop


Epoch 1/10
Epoch 1 training started at: Mon Jan 15 06:56 PM 2024
Total batches: 6390


Training Progress:  10%|█         | 639/6390 [14:57<2:14:30,  1.40s/batch, Loss=2.32]

Batch [639/6390] - Loss: 2.3207 - Progress: 10.00%


Training Progress:  20%|██        | 1278/6390 [29:53<1:59:23,  1.40s/batch, Loss=2.11]

Batch [1278/6390] - Loss: 2.1110 - Progress: 20.00%


...

Training Progress:  30%|███       | 1917/6390 [44:48<1:44:24,  1.40s/batch, Loss=1.69]

Batch [1917/6390] - Loss: 1.6872 - Progress: 30.00%


Training Progress:  40%|████      | 2556/6390 [59:45<1:29:28,  1.40s/batch, Loss=1.93]

Batch [2556/6390] - Loss: 1.9344 - Progress: 40.00%


Training Progress:  49%|████▉     | 3147/6390 [1:13:34<1:15:57,  1.41s/batch, Loss=1.71]